In [1]:
import re
import os
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration

In [2]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
df = pd.read_csv('sports_news_data.csv')
print(df.shape) ; df.head(5)

(9077, 3)


,TITLE,CONTENT,PUBLISH_DT
0,스털링 다이빙 논란 종결?… “오른쪽 다리 접촉 있었잖아”,[스포탈코리아] 유럽축구연맹(UEFA) 유로 2020 심판위원장 로베르토 로세티가 ...,2021-07-15
1,"‘디 마리아 없다’ 유로X코파 베스트11, 이탈리아만 7명",[스포탈코리아] 유로 2020과 코파 아메리카 2021로 베스트11을 만든다면 어떤...,2021-07-15
2,‘슈퍼컴퓨터 예측’ 맨시티 우승-맨유 4위… 토트넘은 ‘6위’,[스포탈코리아] 새 시즌이 시작하기도 전에 슈퍼컴퓨터가 예상한 순위가 나왔다.\n\...,2021-07-15
3,"“이재성, 완벽한 프로… 마인츠서 성공할 것” 킬 디렉터의 애정 듬뿍 응원",[스포탈코리아] 홀슈타인 킬 우베 스토버 디렉터가 이재성을 향해 응원 메시지를 띄웠...,2021-07-15
4,"‘홈킷과 딴판’ 바르사 팬들, NEW 어웨이 셔츠 호평… “가장 좋아하는 색!”",[스포탈코리아] FC 바르셀로나가 새 시즌 원정 유니폼을 공개했다. 팬들은 만족스럽...,2021-07-15


In [4]:
# 결측치 확인
df.isnull().sum()

TITLE         0
CONTENT       2
PUBLISH_DT    0
dtype: int64

In [5]:
# 결측치 제거
df = df.dropna()
df.isnull().sum()

TITLE         0
CONTENT       0
PUBLISH_DT    0
dtype: int64

In [6]:
# 중복값 확인
df[df['CONTENT'].duplicated(keep = False)]

,TITLE,CONTENT,PUBLISH_DT
3456,"첼시, '램파드 후계자' 마운트와 재계약 추진…주급 2배 인상",<p>[골닷컴] 강동훈 기자 = 첼시가 메이슨 마운트(22)와 주급 2배 인상을 조...,2021-10-19
3483,"첼시, '램파드 후계자' 마운트와 재계약 추진…주급 2배 인상",<p>[골닷컴] 강동훈 기자 = 첼시가 메이슨 마운트(22)와 주급 2배 인상을 조...,2021-10-19
3575,"'갑부구단 등극' 뉴캐슬, FFP 준수 위해 129년 만에 역사 바꿀 수도",<p>[골닷컴] 강동훈 기자 = 뉴캐슬 유나이티드가 홈구장의 명칭을 바꿀 수도 있다...,2021-10-22
3578,"'갑부구단 등극' 뉴캐슬, FFP 준수 위해 129년 만에 역사 바꿀 수도",<p>[골닷컴] 강동훈 기자 = 뉴캐슬 유나이티드가 홈구장의 명칭을 바꿀 수도 있다...,2021-10-22
3652,"투헬의 사과 ""홀란드 언급 내 불찰이다""",<p>[골닷컴] 박문수 기자 = 첼시의 토마스 투헬 감독이 친정팀 선수 엘링 홀란드...,2021-10-23
3662,"투헬의 사과 ""홀란드 언급 내 불찰이다""",<p>[골닷컴] 박문수 기자 = 첼시의 토마스 투헬 감독이 친정팀 선수 엘링 홀란드...,2021-10-23
3780,"캐러거 ""프레드, 맥토미니 비판하면서 솔샤르 비판 못 해?""",<p>[골닷컴] 한만성 기자 = '스카이 스포츠' 해설위원 제이미 캐러거가 맨체스터...,2021-10-27
3782,"캐러거 ""프레드, 맥토미니 비판하면서 솔샤르 비판 못 해?""",<p>[골닷컴] 한만성 기자 = '스카이 스포츠' 해설위원 제이미 캐러거가 맨체스터...,2021-10-27
3889,"'골무원은 옛말?' 4경기 침묵 호날두, 토트넘전 기록은?","<p>[골닷컴] 박문수 기자 = 어찌 잘 나가는 듯싶었더니, 주춤하다. 크리스티아누...",2021-10-29
3891,"'골무원은 옛말?' 4경기 침묵 호날두, 토트넘전 기록은?","<p>[골닷컴] 박문수 기자 = 어찌 잘 나가는 듯싶었더니, 주춤하다. 크리스티아누...",2021-10-29


In [7]:
# 중복값 제거
idx = df['CONTENT'].drop_duplicates().index
df = df.loc[idx]
print(df.shape) ; df.head(5)

(9050, 3)


,TITLE,CONTENT,PUBLISH_DT
0,스털링 다이빙 논란 종결?… “오른쪽 다리 접촉 있었잖아”,[스포탈코리아] 유럽축구연맹(UEFA) 유로 2020 심판위원장 로베르토 로세티가 ...,2021-07-15
1,"‘디 마리아 없다’ 유로X코파 베스트11, 이탈리아만 7명",[스포탈코리아] 유로 2020과 코파 아메리카 2021로 베스트11을 만든다면 어떤...,2021-07-15
2,‘슈퍼컴퓨터 예측’ 맨시티 우승-맨유 4위… 토트넘은 ‘6위’,[스포탈코리아] 새 시즌이 시작하기도 전에 슈퍼컴퓨터가 예상한 순위가 나왔다.\n\...,2021-07-15
3,"“이재성, 완벽한 프로… 마인츠서 성공할 것” 킬 디렉터의 애정 듬뿍 응원",[스포탈코리아] 홀슈타인 킬 우베 스토버 디렉터가 이재성을 향해 응원 메시지를 띄웠...,2021-07-15
4,"‘홈킷과 딴판’ 바르사 팬들, NEW 어웨이 셔츠 호평… “가장 좋아하는 색!”",[스포탈코리아] FC 바르셀로나가 새 시즌 원정 유니폼을 공개했다. 팬들은 만족스럽...,2021-07-15


In [8]:
idx = df['TITLE'].drop_duplicates().index
df = df.loc[idx]
print(df.shape) ; df.head(5)

(8995, 3)


,TITLE,CONTENT,PUBLISH_DT
0,스털링 다이빙 논란 종결?… “오른쪽 다리 접촉 있었잖아”,[스포탈코리아] 유럽축구연맹(UEFA) 유로 2020 심판위원장 로베르토 로세티가 ...,2021-07-15
1,"‘디 마리아 없다’ 유로X코파 베스트11, 이탈리아만 7명",[스포탈코리아] 유로 2020과 코파 아메리카 2021로 베스트11을 만든다면 어떤...,2021-07-15
2,‘슈퍼컴퓨터 예측’ 맨시티 우승-맨유 4위… 토트넘은 ‘6위’,[스포탈코리아] 새 시즌이 시작하기도 전에 슈퍼컴퓨터가 예상한 순위가 나왔다.\n\...,2021-07-15
3,"“이재성, 완벽한 프로… 마인츠서 성공할 것” 킬 디렉터의 애정 듬뿍 응원",[스포탈코리아] 홀슈타인 킬 우베 스토버 디렉터가 이재성을 향해 응원 메시지를 띄웠...,2021-07-15
4,"‘홈킷과 딴판’ 바르사 팬들, NEW 어웨이 셔츠 호평… “가장 좋아하는 색!”",[스포탈코리아] FC 바르셀로나가 새 시즌 원정 유니폼을 공개했다. 팬들은 만족스럽...,2021-07-15


In [9]:
def title_cleansing(x):
    new_string = re.sub(r'\[.*\]', '', x)
    return new_string.strip()

def content_cleansing(x):
    new_string = re.sub(r'\n', '', x)
    new_string = re.sub(r'\[스포탈코리아.*\]', '', new_string)
    new_string = re.sub(r'사진=.+$', '', new_string)
    new_string = re.sub(r'.+.기자=', '', new_string)
    new_string = re.sub('\(.*\)|\s-\s.*', '', new_string)
    
    return new_string.strip()

In [10]:
df['TITLE'] = df['TITLE'].progress_apply(lambda x: title_cleansing(x))
df['CONTENT'] = df['CONTENT'].progress_apply(lambda x: content_cleansing(x))

100%|██████████| 8995/8995 [00:46<00:00, 193.49it/s]


In [11]:
df.head(5)

,TITLE,CONTENT,PUBLISH_DT
0,스털링 다이빙 논란 종결?… “오른쪽 다리 접촉 있었잖아”,유럽축구연맹를 주목했다. 수비수가 볼을 터치하지 않았다고 봤다. 멜레의 오른쪽 다리...,2021-07-15
1,"‘디 마리아 없다’ 유로X코파 베스트11, 이탈리아만 7명",유로 2020과 코파 아메리카 2021로 베스트11을 만든다면 어떤 모습일까.지난달...,2021-07-15
2,‘슈퍼컴퓨터 예측’ 맨시티 우승-맨유 4위… 토트넘은 ‘6위’,새 시즌이 시작하기도 전에 슈퍼컴퓨터가 예상한 순위가 나왔다.영국 매체 ‘스포츠 바...,2021-07-15
3,"“이재성, 완벽한 프로… 마인츠서 성공할 것” 킬 디렉터의 애정 듬뿍 응원",홀슈타인 킬 우베 스토버 디렉터가 이재성을 향해 응원 메시지를 띄웠다.이재성은 20...,2021-07-15
4,"‘홈킷과 딴판’ 바르사 팬들, NEW 어웨이 셔츠 호평… “가장 좋아하는 색!”",FC 바르셀로나가 새 시즌 원정 유니폼을 공개했다. 팬들은 만족스럽다는 반응이다.바...,2021-07-15


In [12]:
# !pip install transformers

In [13]:
# seed
seed = 7777
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
# device type
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"# available GPUs : {torch.cuda.device_count()}")
    print(f"GPU name : {torch.cuda.get_device_name()}")
else:
    device = torch.device("cpu")
print(device)

# available GPUs : 1
GPU name : Quadro RTX 6000
cuda


In [16]:
tokenizer = PreTrainedTokenizerFast.from_pretrained('digit82/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('digit82/kobart-summarization').to(device='cuda', non_blocking=True)


In [17]:
text = df['CONTENT'].iloc[5000]
text

'김민재 상당한 금액”이라면서도 “김민재와 손흥민은 대표팀에서 함께 뛴다. 손흥민이 김민재의 프리미어리그 적응을 믿는다면, 도박을 걸어볼 만하다”며 좋게 평가했다.만약 김민재 영입이 성사되지 않더라도 토트넘의 후방 강화는 필수 요소라고 짚었다. 매체는 “김민재가 적임자든 아니든, 콘테 감독은 더 많은 센터백이 필요하다. 다빈손 산체스와 조 로든은 매번 선발로 뛸 수 있다는 것을 증명하지 못했다. 토트넘은 챔피언스리그 진출을 위해 1월 이적시장에서 수비수를 데려와야 한다”고 경고했다.'

In [ ]:
# pytorch>=1.10.0
# transformers==4.16.2
# pytorch-lightning==1.5.10
# streamlit==1.2.0

In [18]:
raw_input_ids = tokenizer.encode(text)
input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]

summary_ids = model.generate(torch.tensor([input_ids]).cuda(),  num_beams=4,  max_length=512,  eos_token_id=1)
tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)

'토트넘은 1월 이적시장에서 수비수를 데려와야 한다며 손흥민이 김민재의 프리미어리그 적응을 믿는다면, 도박을 걸어볼 만하다”며 손흥민 영입이 성사되지 않더라도 토트넘의 후방 강화는 필수 요소라고 짚었다.'

In [19]:
# text_list = df['CONTENT'].values
# df['KoBART'] = [0] * len(text_list)

In [20]:
def KoBART(text):
    try:
        raw_input_ids = tokenizer.encode(text)
        input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]

        summary_ids = model.generate(torch.tensor([input_ids]).cuda(),  num_beams=4,  max_length=512,  eos_token_id=1)
        result = tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)
        return result
    except:
        return "문제"

In [21]:
df['KoBART'] = df['CONTENT'].progress_apply(lambda x: KoBART(x))

100%|██████████| 8995/8995 [01:33<00:00, 96.42it/s] 


In [22]:
df.head()

,TITLE,CONTENT,PUBLISH_DT,KoBART
0,스털링 다이빙 논란 종결?… “오른쪽 다리 접촉 있었잖아”,유럽축구연맹를 주목했다. 수비수가 볼을 터치하지 않았다고 봤다. 멜레의 오른쪽 다리...,2021-07-15,"로세티 위원장이 심판을 대표해 의견을 냈지만, 오심이라고 생각하는 이들이 받아들일지..."
1,"‘디 마리아 없다’ 유로X코파 베스트11, 이탈리아만 7명",유로 2020과 코파 아메리카 2021로 베스트11을 만든다면 어떤 모습일까.지난달...,2021-07-15,유로 2020과 코파 아메리카 2021로 베스트11을 만든다면 어떤 모습일까.
2,‘슈퍼컴퓨터 예측’ 맨시티 우승-맨유 4위… 토트넘은 ‘6위’,새 시즌이 시작하기도 전에 슈퍼컴퓨터가 예상한 순위가 나왔다.영국 매체 ‘스포츠 바...,2021-07-15,시즌이 시작하기도 전에 슈퍼컴퓨터가 예상한 순위가 나왔다.
3,"“이재성, 완벽한 프로… 마인츠서 성공할 것” 킬 디렉터의 애정 듬뿍 응원",홀슈타인 킬 우베 스토버 디렉터가 이재성을 향해 응원 메시지를 띄웠다.이재성은 20...,2021-07-15,"독일 분데스리가 마인츠, 호펜하임을 비롯해 잉글랜드 프리미어리그 크리스탈 팰리스가 ..."
4,"‘홈킷과 딴판’ 바르사 팬들, NEW 어웨이 셔츠 호평… “가장 좋아하는 색!”",FC 바르셀로나가 새 시즌 원정 유니폼을 공개했다. 팬들은 만족스럽다는 반응이다.바...,2021-07-15,FC 바르셀로나가 공식 홈페이지를 통해 2021/2022시즌에 입을 어웨이 킷을 공...


In [25]:
df[df['KoBART'] == '문제']

,TITLE,CONTENT,PUBLISH_DT,KoBART
76,"‘박정인 멀티 골’ 부산, 안산 4-0 완파... 4위 사수",부산 아이파크가 박정인의 멀티 골에 힘입어 안산 그리너스를 완파했다.부산은 17일 ...,2021-07-17,문제
77,‘국대 우승은 처음’… 허벅지에 코파 트로피 새겼다… “가자 아르헨티나!”,"앙헬 디 마리아 챔피언스리그 1회 등 수많은 트로피를 거머쥔 디 마리아지만, 아르헨...",2021-07-17,문제
78,"7년 전 오늘, 역대 최고 공격 라인이 탄생했다",7년 전 7월 17일은 축구 역사상 최고의 공격 라인이 탄생한 날이다. 글로벌 스포...,2021-07-17,문제
79,"‘9번밖에 안 썼잖아’ 모리뉴, 맨유에 손 내밀었지만 거절당해",AS로마의 신임 사령탑 조세 모리뉴 감독이 친정팀 선수를 원했지만 거절당했다.‘메트...,2021-07-17,문제
80,"‘사회생활?’ 스몰링, “모리뉴와 관계 말할 줄 알았어... 타고난 위너”",크리스 스몰링 ‘BBC’를 통해 “감독이 선수에 대해 말하지 않는다는 건 관심이 없...,2021-07-17,문제
...,...,...,...,...
9072,"이제 홈팬 야유 받는 먹튀 선수, 차비 감독 조차 “그만 해라”",FC바르셀로나 스페인 바르셀로나 캄노우에서 열린 아틀레티코 마드리드와 2021/20...,2022-02-07,문제
9073,"성남, 만 17세 유스 김지수와 준프로 계약",성남FC가 만17세 2004년생 수비수 김지수와 준프로 계약을 체결했다. 김지수는 ...,2022-02-07,문제
9074,"오베르마스, 아약스서 쫓겨난다...'여성 동료들에게 부적절한 메시지'",'레전드' 마르크 오베르마스가 아약스에서 쫓겨났다. 이유는 굉장히 굴욕적이었다.아약...,2022-02-07,문제
9075,"바르사 차비 감독, 트라오레 데뷔전 활약에 깜짝...""몸이 야수 같아""",FC 바르셀로나 차비 에르난데스 감독이 데뷔전을 치른 아다마 트라오레를 극찬했다.바...,2022-02-07,문제


In [26]:
new_df = df[df['KoBART'] != '문제']

In [ ]:
text = df.loc[76, 'CONTENT']

In [ ]:
len(text)

In [28]:
new_df.to_pickle('sports_news_KoBART.pkl')